In [1]:
from ultralytics import YOLO
import shutil, os, glob, random, json
import torch


print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA disponible: False


In [2]:
# Ajusta estas rutas a tus carpetas locales:
SOURCE_DIR = r"D:\Proyectos\Salmones\fish_angles_raw"   
DATASET_DIR = r"D:\Proyectos\Salmones\fish_angle_cls"   # se creará con train/val/test

splits = {"train": 0.7, "val": 0.15, "test": 0.15}

# Limpia si ya existía
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
for sp in ["train","val","test"]:
    os.makedirs(os.path.join(DATASET_DIR, sp), exist_ok=True)

# Detecta clases
classes = sorted([d for d in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, d))])
print("Clases detectadas:", classes)

# Crea subcarpetas por split y clase
for sp in ["train","val","test"]:
    for c in classes:
        os.makedirs(os.path.join(DATASET_DIR, sp, c), exist_ok=True)

# Split estratificado simple por carpeta
for c in classes:
    imgs = []
    for ext in ("*.jpg","*.jpeg","*.png"):
        imgs.extend(glob.glob(os.path.join(SOURCE_DIR, c, ext)))
    random.shuffle(imgs)
    n = len(imgs)
    n_train = int(n * splits["train"])
    n_val   = int(n * splits["val"])
    train_files = imgs[:n_train]
    val_files   = imgs[n_train:n_train+n_val]
    test_files  = imgs[n_train+n_val:]

    def copy(files, split):
        for f in files:
            shutil.copy2(f, os.path.join(DATASET_DIR, split, c))
    copy(train_files, "train")
    copy(val_files,   "val")
    copy(test_files,  "test")

sum_train = sum(len(glob.glob(os.path.join(DATASET_DIR,"train",c,"*"))) for c in classes)
sum_val   = sum(len(glob.glob(os.path.join(DATASET_DIR,"val",c,"*"))) for c in classes)
sum_test  = sum(len(glob.glob(os.path.join(DATASET_DIR,"test",c,"*"))) for c in classes)
print(f"Imágenes -> train:{sum_train}  val:{sum_val}  test:{sum_test}")


Clases detectadas: ['0', '135', '180', '225', '270', '315', '45', '90']
Imágenes -> train:760  val:159  test:171


In [3]:
# Construir R (train+val) para Cross-Validation, dejando test como externo ===
import os, shutil, glob

R_DIR = os.path.join(os.path.dirname(DATASET_DIR), "fish_angle_R")
if os.path.exists(R_DIR):
    shutil.rmtree(R_DIR)
os.makedirs(R_DIR, exist_ok=True)

classes = sorted([d for d in os.listdir(os.path.join(DATASET_DIR, "train"))
                  if os.path.isdir(os.path.join(DATASET_DIR, "train", d))])

for c in classes:
    os.makedirs(os.path.join(R_DIR, c), exist_ok=True)

# Si ejecutaste la Celda 3, puedes excluir derivados con sufijos (descomenta para filtrar):
DERIV_SUFFIXES = ["_rot180", "_rot180_"]
def es_derivada(path):
    base = os.path.basename(path)
    return any(suf in base for suf in DERIV_SUFFIXES)

for split in ["train", "val"]:
    for c in classes:
        src = os.path.join(DATASET_DIR, split, c)
        dst = os.path.join(R_DIR, c)
        for ext in ("*.jpg","*.jpeg","*.png"):
            for p in glob.glob(os.path.join(src, ext)):
                if es_derivada(p):
                    continue
                shutil.copy2(p, dst)

def count_in(base):
    return sum(len(glob.glob(os.path.join(base, c, "*"))) for c in classes)

print("Clases:", classes)
print("Total en R (train+val):", count_in(R_DIR))
print("Total en TEST externo :", sum(len(glob.glob(os.path.join(DATASET_DIR, 'test', c, '*'))) for c in classes))


Clases: ['0', '135', '180', '225', '270', '315', '45', '90']
Total en R (train+val): 919
Total en TEST externo : 171


In [4]:
# === K-Fold CV limpio + aumentos controlados ===
import os, glob, shutil, random
from sklearn.model_selection import StratifiedKFold
from PIL import Image

CLASSES   = ['0','45','90','135','180','225','270','315']
FOLDS_DIR = os.path.join(os.path.dirname(DATASET_DIR), "fish_angle_folds")
K = 5

# tope de refuerzo para 90° desde 270° (ej. 0.5 = +50%, 1.0 = +100%)
CAP_RATIO_90 = 1.0

def list_imgs(d):
    files = []
    for ext in ("*.jpg","*.jpeg","*.png"):
        files += glob.glob(os.path.join(d, ext))
    return sorted(files)

def list_imgs_orig(d):
    # Solo originales: evita usar derivados (que contengan "_rot")
    return [p for p in list_imgs(d) if "_rot" not in os.path.basename(p).lower()]

def safe_copy(paths, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    for p in paths:
        shutil.copy2(p, out_dir)

def save_unique(im, dst_dir, base, suffix, ext):
    os.makedirs(dst_dir, exist_ok=True)
    out = os.path.join(dst_dir, f"{base}_{suffix}{ext}")
    i = 1
    while os.path.exists(out):
        out = os.path.join(dst_dir, f"{base}_{suffix}_{i}{ext}")
        i += 1
    im.save(out)

def list_samples_and_labels(base_dir, classes):
    samples, labels = [], []
    for c in classes:
        cdir = os.path.join(base_dir, c)
        for p in list_imgs_orig(cdir):   # <-- SOLO originales
            samples.append(p); labels.append(c)
    return samples, labels

# --- Construye lista de muestras (solo originales) desde R_DIR ---
samples, labels = list_samples_and_labels(R_DIR, CLASSES)

# --- LIMPIA folds previos ---
if os.path.exists(FOLDS_DIR):
    shutil.rmtree(FOLDS_DIR)

skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
fold_specs = []

for k, (train_idx, val_idx) in enumerate(skf.split(samples, labels)):
    fold_root = os.path.join(FOLDS_DIR, f"fold_{k}")
    tr_dir = os.path.join(fold_root, "train")
    va_dir = os.path.join(fold_root, "val")

    # Estructura de carpetas
    for d in [tr_dir, va_dir]:
        for c in CLASSES:
            os.makedirs(os.path.join(d, c), exist_ok=True)

    # Copia SOLO originales a train/ y val/
    train_paths = [samples[i] for i in train_idx]
    val_paths   = [samples[i] for i in val_idx]
    for p in train_paths:
        c = os.path.basename(os.path.dirname(p))
        safe_copy([p], os.path.join(tr_dir, c))
    for p in val_paths:
        c = os.path.basename(os.path.dirname(p))
        safe_copy([p], os.path.join(va_dir, c))

    # Conteos crudos (sin aumentos)
    n90_raw  = len(list_imgs(os.path.join(tr_dir, "90")))
    n270_raw = len(list_imgs(os.path.join(tr_dir, "270")))
    print(f"[fold {k}] RAW train -> 90:{n90_raw}  270:{n270_raw}")

    # Aumentos base: 0 <-> 180 (rotar 180° ambos sentidos) SOLO en train
    # Importante: usar SIEMPRE DONANTES ORIGINALES para evitar cadenas
    for rec, don in [("0","180"), ("180","0")]:
        src = os.path.join(tr_dir, don)
        dst = os.path.join(tr_dir, rec)
        donors = list_imgs_orig(src)
        for f in donors:
            base, ext = os.path.splitext(os.path.basename(f))
            with Image.open(f) as im:
                im2 = im.rotate(180, expand=False)
                save_unique(im2, dst, base, f"from{don}_rot180", ext)

    dst90  = os.path.join(tr_dir, "90")
    src180 = os.path.join(tr_dir, "180")
    cap_add = max(1, int(CAP_RATIO_90 * n90_raw))
    donors180 = list_imgs_orig(src180)
    random.shuffle(donors180)
    to_add = min(cap_add, len(donors180))
    print(f"[fold {k}] 90° crudo={n90_raw}, add from 180: {to_add} (cap={CAP_RATIO_90:.2f}x)")

    for f in donors180[:to_add]:
        base, ext = os.path.splitext(os.path.basename(f))
        with Image.open(f) as im:
            im2 = im.rotate(270, expand=True)
            save_unique(im2, dst90, base, "from180_rot270", ext)

    n90_final = len(list_imgs(dst90))
    print(f"[fold {k}] RESULT train -> 90:{n90_final}")

    fold_specs.append({"root": fold_root, "train": tr_dir, "val": va_dir})
    print(f"Fold {k} listo → {fold_root}")


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# === Entrenar cada fold y promediar métricas ===
from ultralytics import YOLO
import json, os

all_metrics = []

for k, spec in enumerate(fold_specs):
    print(f"\n=== Entrenando fold {k}/{len(fold_specs)-1} ===")
    model = YOLO("yolo11m-cls.pt")
    res = model.train(
        data=spec["root"],
        epochs=50,
        imgsz=256,
        batch=32,
        lr0=0.001,          # ok
        cos_lr=True,
        lrf=0.05,           # <-- sube de 0.01 a 0.05
        optimizer="AdamW",
        patience=15,
        auto_augment=None,
        erasing=0.0,
        workers=0,
        fliplr=0.0, translate=0.0, scale=0.0, mosaic=0.0,
        hsv_h=0.0, hsv_s=0.05, hsv_v=0.05,   # <-- ligero y seguro
        project="runs_cls_cv",
        name=f"fish_angle_y11n_fold{k}"
    )

    # (A) Matriz de confusión del VAL del fold k
    valres = model.val(
        data=spec["root"],   # este root tiene train/ y val/ del fold
        split="val",
        imgsz=256,
        batch=32,
        workers=0,
        plots=True           # <- esto guarda la imagen en la carpeta del experimento del fold
    )


    mdict = {}
    try:
        mdict = res.results_dict
    except Exception:
        pass

    exp_dir = getattr(res, "save_dir", None)
    if exp_dir:
        mj = os.path.join(exp_dir, "results.json")
        if os.path.exists(mj):
            try:
                with open(mj, "r") as f:
                    j = json.load(f)
                if isinstance(j, dict):
                    mdict.update(j)
            except Exception:
                pass

    all_metrics.append(mdict)

def mean_key(key):
    vals = [m[key] for m in all_metrics if key in m]
    return sum(vals)/len(vals) if vals else None

print("\n=== PROMEDIO CV ===")
print("Top-1 acc (mean):", mean_key("metrics/accuracy_top1"))
print("Top-5 acc (mean):", mean_key("metrics/accuracy_top5"))


In [ ]:
# === Entrenamiento final en R (como train y val) y evaluación en test externo ===
import os, glob, shutil
from ultralytics import YOLO

R_DIR = r"D:\Proyectos\Salmones\fish_angle_R"
FINAL_ROOT = r"D:\Proyectos\Salmones\fish_angle_FINAL"
if os.path.isdir(FINAL_ROOT):
    shutil.rmtree(FINAL_ROOT)
for sp in ["train", "val"]:
    os.makedirs(os.path.join(FINAL_ROOT, sp), exist_ok=True)

classes = sorted([d for d in os.listdir(R_DIR) if os.path.isdir(os.path.join(R_DIR, d))])
for sp in ["train", "val"]:
    for c in classes:
        os.makedirs(os.path.join(FINAL_ROOT, sp, c), exist_ok=True)

def place_file(src, dst):
    try: os.link(src, dst)
    except: shutil.copy2(src, dst)

for c in classes:
    src_c = os.path.join(R_DIR, c)
    for ext in ("*.jpg","*.jpeg","*.png"):
        for p in glob.glob(os.path.join(src_c, ext)):
            base = os.path.basename(p)
            place_file(p, os.path.join(FINAL_ROOT, "train", c, base))
            place_file(p, os.path.join(FINAL_ROOT, "val",   c, base))

print("FINAL_ROOT listo:", FINAL_ROOT)

model = YOLO("yolo11m-cls.pt")
res = model.train(
    data=FINAL_ROOT,
    epochs=50,
    imgsz=256,
    batch=32,
    lr0=0.001,
    cos_lr=True,
    lrf=0.05,           # <-- igual que arriba
    optimizer="AdamW",
    patience=15,
    auto_augment=None,
    erasing=0.0,
    workers=0,
    fliplr=0.0, translate=0.0, scale=0.0, mosaic=0.0,
    hsv_h=0.0, hsv_s=0.05, hsv_v=0.05,
    project="runs_cls_final",
    name="fish_angle_y11n_final_R256"
)


DATASET_DIR = r"D:\Proyectos\Salmones\fish_angle_cls"
final_metrics = model.val(
    data=DATASET_DIR,
    split="test",
    imgsz=256,
    batch=32,
    workers=0,
    plots=True
)

print("\n=== MÉTRICAS EN TEST EXTERNO ===")
try:
    print(final_metrics.results_dict)
except:
    pass


In [ ]:
from ultralytics import YOLO
import os, glob, numpy as np
from sklearn.metrics import classification_report

BEST = r"D:\Proyectos\Salmones\runs_cls_final\fish_angle_y11n_final_R2569\weights/best.pt"  # o pon la ruta exacta
DATASET_DIR = r"D:\Proyectos\Salmones\fish_angle_cls"

model = YOLO(BEST)
classes = sorted([d for d in os.listdir(os.path.join(DATASET_DIR,"test")) if os.path.isdir(os.path.join(DATASET_DIR,"test",d))])

y_true, y_pred = [], []
name_to_idx = {c:i for i,c in enumerate(classes)}

for c in classes:
    cdir = os.path.join(DATASET_DIR, "test", c)
    files = []
    for ext in ("*.jpg","*.jpeg","*.png"):
        files += glob.glob(os.path.join(cdir, ext))
    for f in files:
        r = model.predict(f, imgsz=256, verbose=False)
        top1 = int(np.argmax(r[0].probs.data.cpu().numpy()))
        y_true.append(name_to_idx[c]); y_pred.append(top1)

print(classification_report(y_true, y_pred, target_names=classes, digits=4))
